# Interactive Health Coach with Memory using LangMem

This implementation allows users to have a real-time conversation with the health coach via console.


In [ ]:
### Install required libraries

!pip install --upgrade langchain==0.3.18 \
langchain-groq \
mem0ai \
pyautogen \
vecs \
supabase \
langmem==0.0.29 \
langgraph==0.6.4


  Using cached langchain_groq-1.0.0-py3-none-any.whl.metadata (1.7 kB)
INFO: pip is looking at multiple versions of langchain-groq to determine which version is compatible with other requirements. This could take a while.


In [ ]:
### Imports

import os
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langmem import create_manage_memory_tool, create_search_memory_tool
from langgraph.store.memory import InMemoryStore

In [ ]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

In [ ]:
# Set up Groq API
llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0.2, api_key = groq_api_key)

In [ ]:
# Set up memory store
store = InMemoryStore()

In [ ]:
# Create memory tools
manage_memory = create_manage_memory_tool(
    namespace=("health_coach", "user123", "memories"), store=store
)

search_memory = create_search_memory_tool(
    namespace=("health_coach", "user123", "memories"), store=store
)

In [ ]:
# System prompt for the health coach
SYSTEM_PROMPT = """
You are a personal health coach with access to memory tools. Your goal is to help users achieve their fitness goals
by providing personalized advice and tracking their progress over time.

You have two special abilities:
1. You can search past memories about the user to provide personalized advice
2. You can store new information about the user for future reference

When appropriate, you will:
- Search memory for user context (diet, fitness level, goals, etc.)
- Store important new information (changes in weight, new goals, etc.)
- Reference past context to provide continuity

Always be supportive, encouraging, and provide actionable advice.
"""

In [ ]:
def run_interactive_health_coach():

#### -------------Prints a friendly banner and tells you how to leave the chat. -------------


    """
    Interactive console interface for the health coach with memory capabilities
    """
    print("\n" + "=" * 80)
    print("🏃‍♂️ WELCOME TO YOUR PERSONAL HEALTH COACH 🏋️‍♀️")
    print("=" * 80)
    print(
        "I'm your AI health coach with memory. I can provide personalized fitness advice"
    )
    print("and will remember details about your fitness journey between conversations.")
    print("\nType 'exit', 'quit', or 'bye' to end our conversation.\n")




#### ----- Uses a hard-coded user id (in a real app, this would be your account).
#### ----- Starts an empty list to keep a mini history of what you’ve said this session.

    # User ID - in a real app, this would be linked to user accounts
    user_id = "user123"

    # Initialize conversation history
    conversation_history = []





#### ----- Enter the main chat loop. Waits for your message on each turn.

    # Start the conversation loop
    while True:
        # Get user input
        user_message = input("\nYou: ")


#### ----- Exit condition

        # Check if user wants to exit
        if user_message.lower() in ["exit", "quit", "bye"]:
            print(
                "\nThank you for chatting with your health coach! Stay healthy and keep moving!"
            )
            break






#### ----- Search memory for relevant context. Takes what you just said as the search query (truncated to 100 chars).

#### ----- Calls the memory search tool to find past facts about you (diet, goals, injuries, etc.).

#### ----- Why? So the coach can reply with continuity: “Last week you said you’re vegetarian…”





        # Search memory for relevant context
        memory_results = "[]"
        try:
            # Create a search query based on user input and conversation history
            search_query = user_message
            if len(search_query) > 100:
                search_query = search_query[:100]  # Limit query length

            print("\n🔍 Searching memory for relevant context...")
            memory_results = search_memory.invoke({"query": search_query})   ## memory_results is in JSON string format



#### ----- If memories are found

#### ----- If parsing fails, prints a safe snippet of the raw result instead.


            # Parse and pretty-print memory results for debugging
            if memory_results and memory_results != "[]":        ## If this condition is true then memories are found


                try:
                    memory_items = json.loads(memory_results)    ## Converts JSON string to list of dictionaries


                    if memory_items:                             ## If the list contains items → print how many memories were found.
                        print(f"📚 Found {len(memory_items)} relevant memories:")


                        for idx, item in enumerate(memory_items):                      ## Loops through each memory found.
                            if "value" in item and "content" in item["value"]:         ## Each memory object is expected to look something like:  { "value": { "content": "User prefers morning workouts" } }
                                print(
                                    f"  • Memory {idx+1}: {item['value']['content'][:100]}..."     ## It prints the first 100 characters of each memory, so the user sees a preview.
                                )


                    else:                                 ## If the JSON list exists but is empty, print a “no memories” message.
                        print("📭 No relevant memories found.")


                except:             ## If JSON parsing fails, show a warning with the first 100 characters of whatever was returned
                    print(f"⚠️ Memory format: {memory_results[:100]}...")


            else:                  ## If memory_results was empty from the beginning, print “no memories found.”
                print("📭 No previous memories found.")


        except Exception as e:     ## Covers unexpected runtime errors (e.g., memory tool failure).
            print(f"⚠️ Memory search error: {str(e)}")




        # Add to conversation history
        conversation_history.append(user_message)    ## Adds your latest message to a simple in-memory list (this is separate from the memory store).




#### ----- Create context for the model. Keeps only the last 3 user messages as lightweight recent context.


        context = ("\n".join(conversation_history[-3:]) if len(conversation_history) > 1 else user_message)  ## Purpose: give the model a tiny recap without flooding it.





#### ----- Prepare the model prompt (system + human)

        # Generate response with context
        messages = [
            SystemMessage(content=SYSTEM_PROMPT),
            HumanMessage(
                content=f"""
User message: {user_message}

Previous conversation context: {context}

Memory search results: {memory_results}

Respond to the user with personalized health coaching advice based on their message and any relevant information from memory.

Also, identify if there's new important information about the user that should be stored in memory.
Don't mention the memory system directly to the user - they should just experience personalized advice.
"""
            ),
        ]





#### ----- Get the model’s reply.

        # Get response from model
        print("\n⏳ Thinking...")
        response = llm.invoke(messages)    ## Calls the LLM with the prepared messages.

        # Process response to extract potential memory updates
        coach_response = response.content        ## Holds the actual text message the coach will say back to the user.
                                                 ## Later, the program scans coach_response to see if the model included anything like MEMORY: , STORE IN MEMORY:, REMEMBER:
                                                 ## If the AI included one of these markers, the program will extract that memory and save it.
                                                 ## In other words, this sets up the next step where we check if the AI suggested anything new to remember.


        memory_to_store = None                  ## Creates an empty storage slot for possible new memory extracted from the AI response




#### ----- Check if the response contains a clear indication of memory content. The AI coach may return a message like this. Sure! I can help you track your progress. MEMORY: User goal is to lose 5kg by March.

## This code checks whether the reply contains something that the AI wants the system to store as memory, then extracts only that memory part and saves it — without showing that internal section to the user.


        if (
            "MEMORY:" in coach_response
            or "STORE IN MEMORY:" in coach_response
            or "REMEMBER:" in coach_response
        ):                                                  ## Look for memory markers in the AI response



            try:
                # Try to extract memory section - assumes model might format its response with a designated memory section



                for marker in ["MEMORY:", "STORE IN MEMORY:", "REMEMBER:"]:  ## Loop through each possible marker. If the marker (e.g., "MEMORY:") is found, split the text at that point.

                    if marker in coach_response:
                        parts = coach_response.split(marker, 1)

                        if len(parts) > 1:

                            # Extract the memory part and clean up the response
                            memory_section = parts[1].split("\n\n", 1)[0].strip()  ## Pull out the text that comes right after the marker. Result = just the memory text


                            coach_response = coach_response.replace(marker + memory_section, "").strip()  ## Remove the memory from the coach’s visible response. This cleans the message before displaying it to the user


                            memory_to_store = memory_section     ## Store that memory. This variable will later be saved to the memory store.
                            break



            except:    ## If something goes wrong (bad format, unexpected structure), the system won't crash — it will just store a simpler version.

                # If extraction fails, create a general memory
                memory_to_store = f"User message: {user_message}"


        else:         ## If no marker exists
            # Create an automatic memory from this interaction
            memory_to_store = extract_key_information(user_message)




#### ----- Save the new memory (if any)

        # Store memory if we have something to store

        if memory_to_store:

            try:
                print("\n💾 Storing new information in memory...")

                manage_memory.invoke({"content": memory_to_store})      ## Uses the memory management tool to save the extracted info.

                print(f"✅ Stored: {memory_to_store[:50]}..." if len(memory_to_store) > 50 else f"✅ Stored: {memory_to_store}")   ## Prints a small confirmation (or an error if saving fails).


            except Exception as e:
                print(f"⚠️ Memory storage error: {str(e)}")


#### ----- Clean up and show the coach’s message

        # Clean up response to remove any artifacts and display to user

        coach_response = clean_response(coach_response)

        print(f"\nCoach: {coach_response}")



In [ ]:

#### ----- This function runs only when the AI did not explicitly tell us what to store in memory.

#### ----- It takes whatever the user typed and turns it into a generic memory entry.


def extract_key_information(user_message):

    """
    Automatically extract key health/fitness information from user messages
    when no explicit memory instruction is found
    """

    # Simple extraction for demonstration - in a real system, use NLP for better extraction

    memory = f"User shared: {user_message}"

    return memory

In [ ]:

#### ----- clean_response() removes internal system instructions or markers from the AI's output so the user only sees a clean, natural response.


''''

When the AI replies, it might include markers like:

COACH: You should eat more protein.

MEMORY: User prefers morning workouts.

We don’t want the user to see memory-related content, because that’s an internal mechanism. This function scrubs that out before printing the response to the user.


'''





def clean_response(response):


    """Clean up the response to remove any system artifacts"""

    # Remove common formatting markers
    for marker in ["RESPONSE:", "USER RESPONSE:", "COACH:"]:
        if response.startswith(marker):
            response = response[len(marker) :].strip()

    # Remove any remaining memory markers and their content
    for marker in ["MEMORY:", "STORE IN MEMORY:", "REMEMBER:"]:
        if marker in response:
            parts = response.split(marker, 1)
            if len(parts) > 1:
                second_part = parts[1].split("\n\n", 1)
                if len(second_part) > 1:
                    response = parts[0] + second_part[1]
                else:
                    response = parts[0]

    return response.strip()

In [ ]:
if __name__ == "__main__":
    run_interactive_health_coach()


🏃‍♂️ WELCOME TO YOUR PERSONAL HEALTH COACH 🏋️‍♀️
I'm your AI health coach with memory. I can provide personalized fitness advice
and will remember details about your fitness journey between conversations.

Type 'exit', 'quit', or 'bye' to end our conversation.


You: My name is Ankit. I want to get shreadded in 2 months

🔍 Searching memory for relevant context...
📭 No previous memories found.

⏳ Thinking...

💾 Storing new information in memory...
✅ Stored: User shared: My name is Ankit. I want to get shrea...

Coach: Hey Ankit!  
Getting shredded in two months is an ambitious but achievable goal if you’re ready to commit to a focused nutrition, training, and recovery plan. Let’s lay out a roadmap and then fine‑tune it once I know a bit more about where you’re starting from.

---

## 1. Quick‑Start Checklist  
| What I need to know | Why it matters | How it shapes the plan |
|---------------------|----------------|------------------------|
| **Current weight & height** | Sets a baselin